## Test Run 1: Load AbstractNet Dataset: 70K unlabeled (and 2K labeled) Abstracts into DB

This notebook loads the dataset and create labeled *candidates* through labeling function. Feel extra free to document/bring up any upcoming confusion throughout the test run, e.g. are the following two consistent, the **(segment, label) pair** that we want to have, and the **candidates** that we instruct snorkel to extract? 

Before everything, please ensure that you have followed project-level ``README.md`` and installed all python dependencies, e.g. ``tika``.  

We filtered out null abstracts from `ClydeDB.csv` ([AbstractSegmentationCrowdNLP Git repo](https://github.com/zhoujieli/AbstractSegmentationCrowdNLP.git)), resulting in 48,914 valid ones out of 56,851 total abstracts. The 48,914 abstracts are saved to `data/70kpaper.tsv`.



In this section, we preprocess documents by parsing them into *contexts*. *Candidates* are extracted out of *contexts*, which are *instances* (one of the *background*, *mechanism*, *method*, and *findings*).

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os

from snorkel import SnorkelSession
session = SnorkelSession()

# # Here, we just set how many documents we'll process for automatic testing- you can safely ignore this!
n_docs = 500 if 'CI' in os.environ else 1000 #  change the number 1000 to 60,000 for real dataset 

from snorkel.parser import TSVDocPreprocessor

doc_preprocessor = TSVDocPreprocessor('data/70kpaper_061418_cleaned.tsv', encoding="utf-8",max_docs=n_docs)

Get statistics on the number of documents and sentences, as below. This could take 5-8 minutes to load ~60K papers (see progress bar, also might have exception). The following code parses docs into sentences by period, averaging 4.49 sentences per documents. Earlier I spent a few hours debugging some hidden formatting error that confuses Spacey. Need to ensure that we format raw data from .csv into .tsv *without* preceeding and appending quotes.  


In [2]:
from snorkel.parser.spacy_parser import Spacy
from snorkel.parser import CorpusParser


corpus_parser = CorpusParser(parser=Spacy())
%time corpus_parser.apply(doc_preprocessor, count=n_docs)


from snorkel.models import Document, Sentence

print("Documents:", session.query(Document).count())
print("Sentences:", session.query(Sentence).count())

Clearing existing...
Running UDF...
[========================================] 100%

CPU times: user 36.6 s, sys: 1.49 s, total: 38.1 s
Wall time: 25.8 s
Documents: 1000
Sentences: 4487


Next we extract `candidates` by defining the specific `CandidateExtractor` for abstract segmentation. We take *Background* as an example and come back with other segmentation parts, i.e., *mechanism*, *method*, *findings*, later. 

Some more explanation based on my understanding: 
    
1. `Candidates` are defined as a class that contains 1+ `Span` objects within one `Sentence` context.  
    
2. `Span(s)` correspond to conceptual categories in text like people or disease names. 
    
In the intro tutorial example, their `candidate` represents the possible `Spouse` mention `(Barrack Obama, Michelle Obama)`. As readers, we know this mention is true due to external knowledge and the keyword of `wedding` occuring later in the sentence. (Reference: (1) section `Writing a basic CandidateExtractor` in [Intro_tutorial_1](../intro/Intro_Tutorial_1.ipynb); (2) section `Candidate Member Functions and Variables` in [Workshop_1_Snorkel_API](../workshop/Workshop_1_Snorkel_API.ipynb)) 


+ Background: 
  - "Recent research ... ", 
  - "... have/has been widely ...", 
  - "How ... ?" (and as the first sentence), 
  - "Previous work...", 
  - "Motivated by...", 
  - "The success of ...", etc.
+ Mechanism:
  - something
  - some other pattern

We define `CandidateExtractor` as a wrapper of `CandidateSpace` (e.g. `Ngrams` is one type of `CandidateSpace`) and `Matcher` (e.g. `DictionaryMatcher`, `PersonMatcher`). Please make sure that `longest_match_only=True`, since this gets us longest span that contains dictionary words. (Reference: source code [candidates.py](../../snorkel/candidates.py) and [matchers.py](../../snorkel/matchers.py)]

In [3]:
from snorkel.models import candidate_subclass
from snorkel.candidates import Ngrams, CandidateExtractor
from snorkel.matchers import PersonMatcher,DictionaryMatch

Background = candidate_subclass('Background', ['background_cue'])

ngrams = Ngrams(n_max=30)
# Start simple: any ngram that matches the dictionary are *background* candidates! 
dict_matcher=DictionaryMatch(d=['previous','motivated','recent','widely'],longest_match_only=True) 
cand_extractor=CandidateExtractor(Background, [ngrams], [dict_matcher])

Now we apply the defined `CandidateExtractor` to the all `Sentences` in the collection (splitted 90/10/10 for train/dev/test). 

In [4]:
from snorkel.models import Document
from util import number_of_people

docs = session.query(Document).order_by(Document.name).all()

train_sents = set()
dev_sents   = set()
test_sents  = set()

for i, doc in enumerate(docs):
    for s in doc.sentences:
        if number_of_people(s) <= 5:
            if i % 10 == 8:
                dev_sents.add(s)
            elif i % 10 == 9:
                test_sents.add(s)
            else:
                train_sents.add(s)
                
for i, sents in enumerate([train_sents, dev_sents, test_sents]):
    %time cand_extractor.apply(sents, split=i)
    print("Number of candidates extracted:", session.query(Background).filter(Background.split == i).count(),"\n\n")
                



Clearing existing...
Running UDF...
[========================================] 100%

CPU times: user 6.78 s, sys: 267 ms, total: 7.05 s
Wall time: 6.93 s
Number of candidates extracted: 86 


Clearing existing...
Running UDF...
[========================================] 100%

CPU times: user 1.1 s, sys: 106 ms, total: 1.21 s
Wall time: 1.16 s
Number of candidates extracted: 8 


Clearing existing...
Running UDF...
[========================================] 100%

CPU times: user 1.09 s, sys: 112 ms, total: 1.21 s
Wall time: 1.16 s
Number of candidates extracted: 9 




Let's take a look at a few of those extracted `Candidates`! Obviously, since we used `DictionaryMatcher`, all `Candidates` will contain at least one word from the set \['previous','motivated','recent','widely'\].

In [11]:
cands = session.query(Background).filter(Background.split == 0).all()

for i in range(86):
    print("Candidate "+str(i)+"/"+str(len(cands))+": "+str(cands[i].background_cue))
    print("Get Span parent Sentence's text: "+str(cands[i].get_parent().text))
    print()


Candidate 0/86: Span("b'previous'", sentence=4705, chars=[20,27], words=[4,4])
Get Span parent Sentence's text: As described in the previous chapters, some of the most successful work has included road following using CCD cameras [8.10] and cross country navigation using a scanning laser range finder [8.7].

Candidate 1/86: Span("b'recent'", sentence=3535, chars=[25,30], words=[5,5])
Get Span parent Sentence's text: This paper builds off of recent work on rapidly exponentially stabilizing control Lyapunov functions (RES-CLF) and control Lyapunov function based quadratic programs (CLFQP) for underactuated hybrid systems.

Candidate 2/86: Span("b'recent'", sentence=4295, chars=[150,155], words=[23,23])
Get Span parent Sentence's text: Within the sciences of the mind, issues of material embodiment and environmental embedding have emerged as important areas of research attention over recent decades.

Candidate 3/86: Span("b'Recent'", sentence=4670, chars=[0,5], words=[0,0])
Get Span parent

Here comes the question, the `CandidateExtractor` extracts only spans with length 1. But each sentence is matched only once with one span. We would want longer segment, e.g. half part or a whole sentence. 

`Span("b'previous'", sentence=4705, chars=[20,27], words=[4,4])`

In [ ]:



# Get info of sentence
# for sent in (docs[0].sentences):
#     print(sent.__dict__.keys())
#     print(sent.text)
#     print(sent.pos_tags)
#     print(sent.ner_tags)
    

# sents=session.query(Sentence).order_by(Sentence.name).all()
# print(sents[0])

In [2]:
from snorkel.models import candidate_subclass

Spouse = candidate_subclass('Background', ['background'])


from snorkel.candidates import Ngrams, CandidateExtractor
from snorkel.matchers import PersonMatcher

ngrams         = Ngrams(n_max=30)
dict_matcher = DictionaryMatch()
cand_extractor = CandidateExtractor(Spouse, [ngrams], [dict_matcher])
DictionaryMatch()

NameError: name 'DictionaryMatch' is not defined

Thanks for reading! 

Some debugging note at the very end (could ignore). 

```
python -m spacy download en
```

Current issue: parser does not parse *by periods*. Sentence count is significantly fewer than expected! 
Potential fix: https://github.com/explosion/spaCy/issues/93

======= Some more debugging log here (not necessary, could skip reading) ======
~~~~
Xins-MacBook-Pro:~ xin$ source activate snorkel
(snorkel) Xins-MacBook-Pro:~ xin$ python
Python 3.6.4 |Anaconda custom (64-bit)| (default, Jan 16 2018, 12:04:33) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)] on darwin
Type "help", "copyright", "credits" or "license" for more information.
>>> import spacey
Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
ModuleNotFoundError: No module named 'spacey'
>>> import spacy
>>> spacy.load('en')
<spacy.en.English object at 0x1080e1da0>
>>> model=spacy.load('en')
>>> docs=model.tokenizer('Hello, world. Here are two sentences.')
>>> for sent in docs.sents:
...     pritn(sent.text)
... 
Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
  File "spacy/tokens/doc.pyx", line 439, in __get__ (spacy/tokens/doc.cpp:9808)
ValueError: Sentence boundary detection requires the dependency parse, which requires data to be installed. For more info, see the documentation: 
https://spacy.io/docs/usage

>>> for sent in docs.sents:
...     print(sent.text)
... 
Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
  File "spacy/tokens/doc.pyx", line 439, in __get__ (spacy/tokens/doc.cpp:9808)
ValueError: Sentence boundary detection requires the dependency parse, which requires data to be installed. For more info, see the documentation: 
https://spacy.io/docs/usage

>>> from spacy.en import English
>>> nlp = English()
>>> doc = nlp(raw_text)
Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
NameError: name 'raw_text' is not defined
>>> raw_text='Hello, world. Here are two sentences.'
>>> doc = nlp(raw_text)
>>> sentences = [sent.string.strip() for sent in doc.sents]
>>> sentences
['Hello, world.', 'Here are two sentences.']
>>> model(raw_text)
Hello, world. Here are two sentences.
>>> docs=model(raw_text)
>>> docs.sents
<generator object at 0x14ad31948>
>>> docs=model(raw_text)
>>> sentences = [sent.string.strip() for sent in docs.sents]
>>> sentences
['Hello, world.', 'Here are two sentences.']
>>> 
~~~~
